In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.stats import linregress

# Setting path to import Csv files.
auto_insurance_to_load = Path("Resources/AutoInsurance.csv")
car_insurance_claim_to_load = Path("Resources/Car_Insurance_Claim.csv")
insurance_claims_to_load = Path("Resources/insurance_claims.csv")

# Reading CSV files and transforming into Dataframe.
auto_insurance_df = pd.read_csv(auto_insurance_to_load)
big_car_insurance_claim_df = pd.read_csv(car_insurance_claim_to_load)
big_insurance_claims_df = pd.read_csv(insurance_claims_to_load)



In [2]:
auto_insurance_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9135 entries, 0 to 9134
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Customer                       9135 non-null   object 
 1   State                          9135 non-null   object 
 2   Customer Lifetime Value        9135 non-null   float64
 3   Response                       9135 non-null   object 
 4   Coverage                       9135 non-null   object 
 5   Education                      9135 non-null   object 
 6   Effective To Date              9135 non-null   object 
 7   EmploymentStatus               9135 non-null   object 
 8   Gender                         9135 non-null   object 
 9   Income                         9135 non-null   int64  
 10  Location Code                  9135 non-null   object 
 11  Marital Status                 9135 non-null   object 
 12  Monthly Premium Auto           9135 non-null   i

In [63]:
unclean_auto_df =  auto_insurance_df[[
                            "State",
                            "Coverage",
                            "Education",
                            "Income",
                            "Months Since Policy Inception",
                            "Months Since Last Claim",
                            "Number of Policies",
                            "Marital Status",
                            "EmploymentStatus",
                            "Policy Type",
                            "Monthly Premium Auto",
                            "Total Claim Amount",
                            "Vehicle Class", 
                            "Vehicle Size"
                            ]]

auto_df = unclean_auto_df.drop(index=0)

income_df = auto_df.loc[auto_df["Income"] != 0]
income_df[["Income","Monthly Premium Auto","Total Claim Amount","Number of Policies","Months Since Policy Inception","Months Since Last Claim"]].corr()

,Income,Monthly Premium Auto,Total Claim Amount,Number of Policies,Months Since Policy Inception,Months Since Last Claim
Income,1.000000,-0.012372,-0.221879,-0.001324,0.015799,-0.034453
Monthly Premium Auto,-0.012372,1.000000,0.636084,-0.026795,0.020112,0.005887
Total Claim Amount,-0.221879,0.636084,1.000000,-0.012253,-0.003733,0.001460
Number of Policies,-0.001324,-0.026795,-0.012253,1.000000,-0.020548,0.017022
Months Since Policy Inception,0.015799,0.020112,-0.003733,-0.020548,1.000000,-0.039993
Months Since Last Claim,-0.034453,0.005887,0.001460,0.017022,-0.039993,1.000000
